In [1]:
import scqubits as scqubits

import jax.numpy as jnp
from jax.scipy.optimize import minimize
import jax.scipy as sci
import qutip as qt
import matplotlib.pyplot as plt
import os
import dynamiqs as dq
from swt import swt_subspace
# Set JAX NumPy precision to float64
import jax
try:
    from jax.config import config
except ImportError:
    # Fallback for older JAX versions
    config = jax.config
config.update("jax_enable_x64", True)

# Disable JAX multi-threading to avoid issues with fork
os.environ["XLA_FLAGS"] = "--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable GPU if present
# Set thread count to 1 to disable parallelism
config.update('jax_platforms', 'cpu')  # Force CPU backend
config.update('jax_disable_jit', True)  # Disable JIT compilation

class Hamiltonian:
    def __init__(self, phi_ex, Ej, Ec, bare_dim, trunc_dim, omega_c1, omega_c2, phizpf_dependence = True, twolevelapp = False):
        self.phi_ex = phi_ex
        self.Ej = Ej * 2*jnp.pi
        self.Ec = Ec * 2*jnp.pi
        self.beta = 1
        self.N = 1
        self.original_dim = bare_dim
        self.trunc_dim = trunc_dim
        self.sdim, self.c2dim, self.c1dim = bare_dim
        self.total_dim = self.sdim*self.c2dim*self.c1dim
        self.omega_c1 = omega_c1
        self.omega_c2 = omega_c2
        self.phizpf_dependence = phizpf_dependence
        self.twolevelapp = twolevelapp
        self.g = 0.05*2*jnp.pi
        self.option = dq.Options(progress_meter=None)
        self.H , self.H_control = self.get_H()
        self.H_dressed, self.H_control_dressed = self.dressed_basis()
        

    def sort_eigenpairs(self, eigenvalues, eigenvectors):
        n = eigenvectors.shape[0]
        sorted_indices = []

        for i in range(n):
            max_abs_vals = jnp.abs(eigenvectors[i, :])
            max_index = jnp.argmax(max_abs_vals)
            while max_index in sorted_indices:
                max_abs_vals = max_abs_vals.at[max_index].set(-jnp.inf)
                max_index = jnp.argmax(max_abs_vals)
            sorted_indices.append(max_index)

        sorted_indices_array = jnp.array(sorted_indices)
        sorted_eigenvalues = eigenvalues[sorted_indices_array]
        sorted_eigenvectors = eigenvectors[:, sorted_indices_array]
        return sorted_eigenvalues, sorted_eigenvectors
    
    def state_index(self, index,dim):
        n,k,m = index
        N,K,M = dim
        return n*K*M + k*M + m
    
    def get_indices(self, trunc):
        """
        Convert a list of 3D indices to 1D indices based on given dimensions.
        indecies follow (squid, cavity2, cavity1)
        
        Args:
            trunc: List of three integers [N,K,M] specifying the dimensions to include
                 N is number of SNAIL states
                 K is number of cavity 2 states
                 M is number of cavity 1 states
                 
        Returns:
            List of 1D indices corresponding to the 3D indices
        """
        N, K, M = trunc
        indices_1d = []
        
        # Generate all 3D indices and store indices outside truncation
        for n in range(self.sdim):
            for k in range(self.c2dim):
                for m in range(self.c1dim):
                    # Only add indices that are outside the truncation
                    if n >= N or k >= K or m >= M:
                        index = [n,k,m]
                    
                        # Convert to 1D index using state_index method
                        idx = self.state_index(index, self.original_dim)
                        indices_1d.append(idx)
        return indices_1d
        
    def remove_row_col(self, matrix, indices):
        # Check if matrix is square
        if matrix.shape[0] != matrix.shape[1]:
            raise ValueError("Input matrix must be square")
        
        # Convert single index to list if needed
        if isinstance(indices, (int, jnp.integer)):
            indices = [indices]
            
        # Convert to list if numpy array
        if isinstance(indices, jnp.ndarray):
            indices = indices.tolist()
            
        # Sort indices in descending order to avoid changing indices when deleting
        indices = sorted(indices, reverse=True)
        
        # Check if indices are valid
        if any(i < 0 or i >= matrix.shape[0] for i in indices):
            raise ValueError(f"All indices must be between 0 and {matrix.shape[0]-1}")
        
        # Remove rows and columns
        result = matrix.copy()
        for idx in indices:
            result = jnp.delete(jnp.delete(result, idx, axis=0), idx, axis=1)
            
        return result
    
    def truncate_matrix(self, matrix, trunc):
        indices = self.get_indices(trunc)
        return self.remove_row_col(matrix, indices)
    
    def annihilation(self, dim):
        return jnp.diag(jnp.sqrt(jnp.arange(1,dim)),1)
    def creation(self, dim):
        return jnp.diag(jnp.sqrt(jnp.arange(1,dim)),-1)
    def tensor_product(self,A,B,C):
        return jnp.kron(A,jnp.kron(B,C))

    def cosm(self, matrix):
        return (sci.linalg.expm(1j*matrix) + sci.linalg.expm(-1j*matrix))/2
    
    def SNAIL(self):
        """
        This function starts from SNAIL hamiltonian in oscillator basis,  and then calculates the SNAIL Hamiltonian and returns the Hamiltonian and the charge operator in eigenbasis. 
        returns: Hs, charge_op in eigenbasis
        """
        beta = self.beta
        N = self.N
        Ej = self.Ej
        Ec = self.Ec
        phi_ex = self.phi_ex*2*jnp.pi
        sdim = 40
        # Find minimum of SNAIL potential
        def U_s(phi): 
            # Need to ensure the function returns a scalar for JAX gradient
            return jnp.asarray(-beta*jnp.cos(phi-phi_ex)-N*jnp.cos(phi/N))[0]
        
        phi_min = minimize(U_s, jnp.array([0.]), method='BFGS').x
        # potential expansion around minimum
        c2 = beta*jnp.cos(phi_min - phi_ex) + 1/N*jnp.cos(phi_min/N)
        omega_s = jnp.sqrt(8*c2*Ej*Ec)
        if self.phizpf_dependence == True:
            phi_zpf = jnp.power(2*Ec/(Ej*c2),1/4)
        else:
            phi_zpf = jax.lax.stop_gradient(jnp.power(2*Ec/(Ej*c2),1/4))
        self.phi_zpf = phi_zpf
        g2 = Ej*phi_zpf**2*c2/2
        s = self.annihilation(sdim)
        sd = self.creation(sdim)
        x2 = jnp.matmul(s+sd,s+sd)
        Hs =( omega_s * jnp.matmul(sd,s)
            - Ej*(beta*self.cosm(phi_zpf*(s+sd)+(phi_min-phi_ex)*jnp.identity(sdim))
            + N*self.cosm((phi_zpf*(s+sd)+phi_min*jnp.identity(sdim))/N))- g2*x2)
        charge_op = -1j*(s-sd)/2/phi_zpf
        energy0,U = jnp.linalg.eigh(Hs)
        energy0,U = self.sort_eigenpairs(energy0, U)
        energy0 = energy0 - energy0[0]
        Ud = U.transpose().conjugate()
        Hs = Ud@Hs@U
        charge_op = Ud@charge_op@U
        Hs = Hs - Hs[0,0]*jnp.identity(sdim)
        Hs = Hs[0:self.sdim,0:self.sdim]
        charge_op = charge_op[0:self.sdim,0:self.sdim]
        return Hs,charge_op
    
    def get_H(self):
        """This function calculates the composite system Hamiltonian and returns the Hamiltonian and the SNAIL charge operator in composite Hilbert space. Returned matrix is under eigenbasis of each subsystem
        returns: H = Hc1 + Hc2 + Hs + H_int1 + H_int2, charge_op in composite Hilbert space
        """
        c1dim = 20
        c2dim = 20

        Hs,charge_op, = self.SNAIL()
        squid = [Hs,charge_op]
        cavity = scqubits.Oscillator(
            E_osc=self.omega_c1,
            truncated_dim = c1dim
        )
        Hc = jnp.diag(jnp.array(cavity.eigenvals(evals_count=c1dim) - cavity.eigenvals(evals_count=c1dim)[0]))*2*jnp.pi
        Hc = Hc[0:self.c1dim,0:self.c1dim]
        Vc = jnp.array(cavity.creation_operator() + cavity.annihilation_operator())
        Vc = Vc[0:self.c1dim,0:self.c1dim]
        cavity = [Hc,Vc]

        cavity2 = scqubits.Oscillator(
            E_osc=self.omega_c2,
            truncated_dim = c2dim
        )
        Hc2 = jnp.diag(jnp.array(cavity2.eigenvals(evals_count=c2dim) - cavity2.eigenvals(evals_count=c2dim)[0]))*2*jnp.pi
        Hc2 = Hc2[0:self.c2dim,0:self.c2dim]
        Vc2 = jnp.array(cavity2.creation_operator() + cavity2.annihilation_operator())
        Vc2 = Vc2[0:self.c2dim,0:self.c2dim]
        cavity2 = [Hc2,Vc2]
        H,H_control = self.composite_sys(squid,cavity,cavity2)
        return H,H_control

    def composite_sys(self, squid,cavity, cavity2,):
        Hs, charge_op = squid
        Hc, Vc = cavity
        Hc2, Vc2 = cavity2
        sdim = Hs.shape[0]
        cdim = Hc.shape[0]
        cdim2 = Hc2.shape[0]
        Ic = jnp.identity(cdim)
        Ic2 = jnp.identity(cdim2)
        Is = jnp.identity(sdim)
        Hs = self.tensor_product(Hs,Ic,Ic2)
        Hc = self.tensor_product(Is, Hc, Ic2)
        Hc2 = self.tensor_product(Is, Ic, Hc2)
        H_int = self.g * self.tensor_product(charge_op, Vc, Ic2) 
        H_int2 = self.g * self.tensor_product(charge_op, Ic, Vc2) 
        H = Hs + Hc + Hc2 + H_int + H_int2
        H_control = self.tensor_product(charge_op, Ic, Ic2)
        return H, H_control

    def dressed_basis(self):
        H , H_control = self.get_H()
        evals,U = jnp.linalg.eigh(H)
        evals,U = self.sort_eigenpairs(evals,U)
        Ud = U.transpose().conjugate()
        H_dressed = Ud@H@U
        H_dressed = H_dressed - H_dressed[0,0]*jnp.identity(self.total_dim)
        H_control_dressed = Ud@H_control@U
        return H_dressed, H_control_dressed
    
    def keep_same_elements(self, target, given):
        """
        Modifies the given matrix to have non-zero elements only where the target matrix has non-zero elements.
        
        Args:
            target: The reference matrix that defines the non-zero pattern
            given: The matrix to be modified
            
        Returns:
            A modified version of the given matrix with zeros where target has zeros
        """
        # Create a mask where target is non-zero
        mask = jnp.abs(target) > 1e-10
        # Apply the mask to the given matrix
        return jnp.where(mask, given, 0.0)
    
    def HR(self, A, omegad):
        H0, Hc = self.H_dressed, self.H_control_dressed
        s = self.annihilation(self.sdim)
        sd = self.creation(self.sdim)
        Ic1 = jnp.identity(self.c1dim)
        Ic2 = jnp.identity(self.c2dim)
        H0 = H0 - self.tensor_product(omegad*sd@s, Ic1, Ic2)
        target = self.tensor_product(sd+s, Ic1, Ic2)
        Hc = self.keep_same_elements(target, Hc)
        if self.twolevelapp == True:
            H0 = self.truncate_matrix(H0, [2, self.trunc_dim[1], self.trunc_dim[2]])
            Hc = self.truncate_matrix(Hc, [2, self.trunc_dim[1], self.trunc_dim[2]])
        else:
            H0 = self.truncate_matrix(H0, self.trunc_dim)
            Hc = self.truncate_matrix(Hc, self.trunc_dim)
        self.H_dressed_truncated = H0
        self.Hcapp = Hc
        H = H0 + A/2*Hc
        eigvals, _ = jnp.linalg.eigh(H)
        eigvals, _ = self.sort_eigenpairs(eigvals, _)
        eigvals = eigvals - eigvals[0]
        return eigvals
    
    def approximateder(self, A=0.0, omegad=0.0):
        """
        Calculate the derivative of the rotating frame Hamiltonian energies with respect to phi_ex
        
        Args:
            A: Drive amplitude
            omegad: Drive frequency
            
        Returns:
            Array of derivatives for each energy level
        """
        # Define a function that computes energy levels for a given phi
        def energy_levels(phi):
            sc = Hamiltonian(phi, self.Ej/(2*jnp.pi), self.Ec/(2*jnp.pi), 
                            self.original_dim, self.trunc_dim, self.omega_c1, 
                            self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            return sc.HR(A, omegad)
        
        # Compute derivatives and flatten the result
        return jax.jacobian(energy_levels)(self.phi_ex).reshape(-1)
    
    def floquet_eigenvectors(self, A=0e-3*2*jnp.pi, omega=6.185*2*jnp.pi):
        """
        Calculate the Floquet eigenvectors for the system.
        
        Args:
            A: Drive amplitude
            omega: Drive frequency
            
        Returns:
            Eigenvectors of the Floquet propagator
        """
        H0, Hc = self.H_dressed, self.H_control_dressed
        H0 = dq.asqarray(H0)
        Hc = dq.asqarray(Hc)
        T = (2 * jnp.pi) / omega
        tsave = jnp.linspace(0, T, 10)
        # Define the Hamiltonian
        def cos_drive(t):
            return A * jnp.cos(omega * t)
        H = H0 + dq.modulated(cos_drive, Hc)
        result = dq.sepropagator(H, tsave, options=self.option)
        U = result.final_propagator
        # Only compute and return eigenvectors
        evecs = U._eig()[1]
        return evecs
    
    def quasi_energy(self, evecs, A=0e-3*2*jnp.pi, omega=6.185*2*jnp.pi):
        """
        Calculate the quasi-energies of the system.
        
        Args:
            evecs: Eigenvectors from floquet_eigenvectors
            A: Drive amplitude
            omega: Drive frequency
            
        Returns:
            Quasi-energies of the system
        """
        # Rebuild the propagator to get eigenvalues only
        H0, Hc = self.H_dressed, self.H_control_dressed
        H0 = dq.asqarray(H0)
        Hc = dq.asqarray(Hc)
        T = (2 * jnp.pi) / omega
        tsave = jnp.linspace(0, T, 10)
        def cos_drive(t):
            return A * jnp.cos(omega * t)
        H = H0 + dq.modulated(cos_drive, Hc)
        result = dq.sepropagator(H, tsave, options=self.option)
        U = result.final_propagator
        evals = U._eigvals()
        evals, evecs = self.sort_eigenpairs(evals, evecs.to_jax())
        e_quasi = -jnp.angle(evals)/T
        e_quasi = e_quasi - e_quasi[0] * jnp.ones(len(e_quasi))
        if self.trunc_dim[1] == 1:
            return e_quasi[1], e_quasi[2], e_quasi[3]
        else:
            return e_quasi[1], e_quasi[self.trunc_dim[2]]
    
    def equasi_gradient(self, A, omega):
        """
        Compute the gradient of quasi-energy with respect to phi_ex
        
        Args:
            A: Drive amplitude
            omega: Drive frequency
            
        Returns:
            Gradient of quasi-energy with respect to phi_ex
        """
        # Compute eigenvectors once and detach them from the computational graph
        evecs = jax.lax.stop_gradient(self.floquet_eigenvectors(A, omega))
        
        def quasi_energy_levels(phi):
            sc = Hamiltonian(phi, self.Ej/(2*jnp.pi), self.Ec/(2*jnp.pi), 
                               self.original_dim, self.trunc_dim, self.omega_c1, 
                               self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            
            # Use the detached eigenvectors to compute quasi-energy
            return sc.quasi_energy(evecs, A, omega)
        
        # Compute the Jacobian with respect to phi_ex and flatten the result
        return jax.jacobian(quasi_energy_levels)(self.phi_ex)
    
    def analytical_app(self, A, omegad):
        """
        Compute the derivative of the Schrieffer-Wolff transformed energies with respect to phi_ex.

        Args:
            A: Drive amplitude
            omegad: Drive frequency

        Returns:
            Array of derivatives for each energy level.
        """
        def transformation(phi):
            sc = Hamiltonian(phi, self.Ej/(2*jnp.pi), self.Ec/(2*jnp.pi),
                               self.original_dim, self.trunc_dim, self.omega_c1,
                               self.omega_c2, self.phizpf_dependence, self.twolevelapp)
            sc.HR(A, omegad)
            H0 = sc.H_dressed_truncated
            V = sc.Hcapp
            return swt_subspace(H0, A / 2 * V)

        return jax.jacobian(transformation)(self.phi_ex)


In [2]:
Ej = 30.19
Ec = 0.1
omega_c1 = 5.226
omega_c2 = 8.135
phi_ex = 0.2
bare_dim = [10,1,6]
trunc_dim = [5,1,4]
phizpf_dependence = True
twolevelapp = False
sc = Hamiltonian(phi_ex, Ej, Ec, bare_dim, trunc_dim, omega_c1, omega_c2, phizpf_dependence, twolevelapp)
A = 1e-3*2*jnp.pi
omegad = 6.169*2*jnp.pi
sc.HR(A, omegad)
H0 = sc.H_dressed_truncated
V = sc.Hcapp
sc.analytical_app(A, omegad)

Array([  -0.        ,   -0.27486953,   -0.60763266,   -0.97136362,
        -43.56809813,  -44.32698414,  -45.03505464,  -45.71936908,
        -88.22580613,  -89.20600208,  -90.20217918,  -91.21398666,
       -132.15860305, -133.77959302, -135.45299136, -137.2139708 ,
       -175.51392443, -178.35135194, -181.38190158, -184.90275462],      dtype=float64, weak_type=True)

In [97]:
dE_dphi

Array([  -0.        ,   -0.27486953,   -0.60763266,   -0.97136362,
        -43.56809813,  -44.32698414,  -45.03505464,  -45.71936908,
        -88.22580613,  -89.20600208,  -90.20217918,  -91.21398666,
       -132.15860305, -133.77959302, -135.45299136, -137.2139708 ,
       -175.51392443, -178.35135194, -181.38190158, -184.90275462],      dtype=float64, weak_type=True)

Array([ 6.01612008e-04+2.15315775e-18j,  4.95339562e-04+1.16280683e-17j,
        4.20474377e-04+3.38813179e-21j,  3.64885637e-04-2.53703308e-17j,
       -4.96892539e-04-2.18873314e-18j, -3.93595876e-04-1.16280683e-17j,
       -3.21584490e-04-1.69406589e-21j, -2.68732980e-04+2.53703308e-17j,
       -2.50123121e-05+3.45165926e-20j, -2.40109708e-05+5.48586182e-21j,
       -2.30957247e-05-1.27054942e-21j, -2.22781892e-05+2.46830695e-21j,
       -1.00639737e-05-7.87889533e-23j, -1.02382180e-05-1.54848211e-21j,
       -1.04634285e-05+4.00665609e-25j, -1.07304197e-05-4.03664139e-22j,
       -6.96431830e-05+2.83309360e-23j, -6.74944970e-05+5.16987883e-26j,
       -6.53307336e-05-3.10192730e-25j, -6.31440487e-05-3.11019910e-22j],      dtype=complex128)